In [1]:

%pprint
%matplotlib inline
import sys
if (osp.join(os.pardir, 'py') not in sys.path): sys.path.insert(1, osp.join(os.pardir, 'py'))
from jobpostlib import (crf, cu, datetime, duration, hau, hc, humanize, ihu, lru, nu, osp, scrfcu, slrcu, ssgdcu, su, t0, time, wsu, speech_engine)
from pandas import DataFrame
import re

Pretty printing has been turned OFF
======== Neo4j/4.4.7 ========
Utility libraries created in 5 seconds


In [3]:

def get_pos_and_consecutives(file_name):
    child_strs_list = ha.get_navigable_children_from_file(file_name)
    child_tags_list = ha.construct_child_tags_list(child_strs_list)
    feature_dict_list = cu.query_feature_dict_list(child_tags_list, child_strs_list)
    feature_tuple_list = [
        hc.get_feature_tuple(feature_dict, pos_lr_predict_single=None, pos_crf_predict_single=None, pos_sgd_predict_single=None)
        for feature_dict in feature_dict_list
    ]
    pos_symbol_predictions_list = ea.CRF.predict_single(ea.sent2features(feature_tuple_list))
    pos_list = []
    for pos, feature_tuple in zip(pos_symbol_predictions_list, feature_tuple_list):
        navigable_parent = feature_tuple[1]
        if navigable_parent in ha.NAVIGABLE_PARENT_IS_HEADER_DICT:
            pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list)
        else:
            pos_list.append(pos)
    consecutives_list = []
    for pos, v in groupby(pos_list):
        consecutives_count = len(list(v))
        consecutives_tuple = (pos, consecutives_count)
        consecutives_list.append(consecutives_tuple)
    
    return consecutives_list, pos_list, child_strs_list

In [4]:

TITLE_SEPARATOR = ' - '
PARENTH_REGEX = re.compile(r'[\(\[][^\)\]]+[\)\]]')
def get_extracted_posting_location(file_name):
    title_str = ' '.join(file_name.split('_')[:-1])
    title_str = re.sub(' - *| *- ', TITLE_SEPARATOR, title_str)
    matched_text_list = []
    for match_obj in PARENTH_REGEX.finditer(title_str):
        matched_text = match_obj.group()
        matched_text_list.append(matched_text)
    for matched_text in matched_text_list:
        replacement_text = matched_text.replace(TITLE_SEPARATOR, '-')
        replacement_text = replacement_text[1:-1].strip()
        replacement_text = replacement_text.replace('. ', TITLE_SEPARATOR).replace(' + ', TITLE_SEPARATOR)
        replacement_text = TITLE_SEPARATOR + replacement_text + TITLE_SEPARATOR
        title_str = title_str.replace(matched_text, replacement_text)
    posting_title_list = title_str.split(TITLE_SEPARATOR)
    posting_title_list = [posting_title_str.strip() for posting_title_str in posting_title_list]
    posting_title_list = list(filter(len, posting_title_list))
    extracted_posting_location = posting_title_list.pop()
    if extracted_posting_location == 'Indeed.com':
        extracted_posting_location = posting_title_list.pop()
    
    return extracted_posting_location, posting_title_list

In [5]:

REMOTE_STRS_LIST = ['remote', 'telecommute', 'home-based', 'wfh', 'work from home']
QUAL_STRS_LIST = ['citizen', 'clearance', 'usc', 'gc', 'ead']
US_STRS_LIST = ['united states', 'us', 'usa', 'us citizens only']
PAY_STRS_LIST = ['b2b', 'up to $']
DURATION_STRS_LIST = ['direct hire']
def create_details_lists(posting_title_list):
    EXTRACTED_LOCATION_LIST = []
    EXTRACTED_REQ_QUALS_LIST = []
    extracted_job_title_list = []
    EXTRACTED_PAY_LIST = []
    duration_details = []
    while len(posting_title_list):
        posting_title_str = posting_title_list.pop()
        if any(map(lambda x: x in posting_title_str.lower(), REMOTE_STRS_LIST)):
            EXTRACTED_LOCATION_LIST.append(posting_title_str)
        elif any(map(lambda x: re.search(f'\\b{x}\\b', posting_title_str.lower()), QUAL_STRS_LIST)):
            EXTRACTED_REQ_QUALS_LIST.append(posting_title_str)
        elif any(map(lambda x: x == posting_title_str.lower(), US_STRS_LIST)):
            EXTRACTED_LOCATION_LIST.append(posting_title_str)
        elif any(map(lambda x: x in posting_title_str.lower(), PAY_STRS_LIST)):
            EXTRACTED_PAY_LIST.append(posting_title_str)
        elif any(map(lambda x: x in posting_title_str.lower(), DURATION_STRS_LIST)):
            duration_details.append(posting_title_str)
        else:
            posting_title_str = posting_title_str.strip()
            if posting_title_str.startswith(','):
                posting_title_str = posting_title_str[1:].strip()
            extracted_job_title_list.append(posting_title_str)
    
    return EXTRACTED_LOCATION_LIST, EXTRACTED_REQ_QUALS_LIST, EXTRACTED_PAY_LIST, duration_details, extracted_job_title_list

In [6]:

COMMAS_REGEX = re.compile(r', ([^,]+),')
REPLACEMENT_STR = f'{TITLE_SEPARATOR}\\1{TITLE_SEPARATOR}'
def extract_job_title_info(file_name):
    extracted_posting_location, posting_title_list = get_extracted_posting_location(file_name)
    commas_list = []
    while len(posting_title_list):
        posting_title_str = posting_title_list.pop()
        posting_title_str = re.sub(COMMAS_REGEX, REPLACEMENT_STR, posting_title_str)
        if any(map(lambda x: x in posting_title_str.lower(), REMOTE_STRS_LIST)):
            posting_title_str = re.sub(f"-({'|'.join(REMOTE_STRS_LIST)})", f'{TITLE_SEPARATOR}\\1', posting_title_str, 0, re.IGNORECASE)
        commas_list.append(posting_title_str)
    title_str = TITLE_SEPARATOR.join(commas_list)
    posting_title_list = title_str.split(TITLE_SEPARATOR)
    assert len(posting_title_list), f'{title_str}: {title_str.split(TITLE_SEPARATOR)}'
    (EXTRACTED_LOCATION_LIST, EXTRACTED_REQ_QUALS_LIST, EXTRACTED_PAY_LIST, duration_details,
     extracted_job_title_list) = create_details_lists(posting_title_list)
    
    return (extracted_posting_location, EXTRACTED_LOCATION_LIST, EXTRACTED_REQ_QUALS_LIST, EXTRACTED_PAY_LIST, duration_details,
            extracted_job_title_list)

In [7]:

def get_section_list(pos):
    if pos == 'H-JD':
        return EXTRACTED_DURATION_LIST
    elif pos == 'H-ER':
        return EXTRACTED_EDUCATION_LIST
    elif pos == 'H-SP':
        return EXTRACTED_PAY_LIST
    elif pos == 'H-PQ':
        return EXTRACTED_PREF_QUALS_LIST
    elif pos == 'H-TS':
        return EXTRACTED_TASK_SCOPE_LIST
    elif pos == 'H-OL':
        return EXTRACTED_LOCATION_LIST
    elif pos == 'H-IP':
        return EXTRACTED_INTERVIEW_LIST
    elif pos == 'H-LN':
        return EXTRACTED_LEGAL_LIST
    elif pos == 'H-RQ':
        return EXTRACTED_REQ_QUALS_LIST
    elif pos == 'H-CS':
        return EXTRACTED_CORP_LIST
    elif pos == 'H-O':
        return EXTRACTED_OTHER_LIST
    
    return []

In [8]:

import random

files_list = os.listdir(ha.SAVES_HTML_FOLDER)
file_name = random.choice(files_list)

In [9]:

EXTRACTED_EDUCATION_LIST = []
EXTRACTED_PREF_QUALS_LIST = []
EXTRACTED_TASK_SCOPE_LIST = []
EXTRACTED_INTERVIEW_LIST = []
EXTRACTED_LEGAL_LIST = []
EXTRACTED_CORP_LIST = []
EXTRACTED_OTHER_LIST = []
(extracted_posting_location, EXTRACTED_LOCATION_LIST, EXTRACTED_REQ_QUALS_LIST, EXTRACTED_PAY_LIST, EXTRACTED_DURATION_LIST,
 extracted_job_title_list) = extract_job_title_info(file_name)
def get_pos_list(file_name):
    child_strs_list = ha.get_navigable_children_from_file(file_name)
    child_tags_list = ha.construct_child_tags_list(child_strs_list)
    feature_dict_list = cu.query_feature_dict_list(child_tags_list, child_strs_list)
    feature_tuple_list = [hc.get_feature_tuple(feature_dict, pos_lr_predict_single=None, pos_crf_predict_single=None, pos_sgd_predict_single=None) for feature_dict in feature_dict_list]
    pos_symbol_predictions_list = ea.CRF.predict_single(ea.sent2features(feature_tuple_list))
    pos_list = []
    for pos, feature_tuple in zip(pos_symbol_predictions_list, feature_tuple_list):
        navigable_parent = feature_tuple[1]
        if navigable_parent in ha.NAVIGABLE_PARENT_IS_HEADER_DICT:
            pos_list = cu.append_parts_of_speech_list(navigable_parent, pos_list)
        else:
            pos_list.append(pos)
    
    return pos_list, child_strs_list

In [10]:

from itertools import groupby

#for file_name in files_list:
pos_list, child_strs_list = get_pos_list(file_name)
consecutives_list = []
for key_pos, consecutive_value in groupby(pos_list):
    value_list = list(consecutive_value)
    consecutives_count = len(value_list)
    consecutives_tuple = (key_pos, consecutives_count)
    consecutives_list.append(consecutives_tuple)
for pos in list(set(pos_list) - set(['O'])):
    section_list = get_section_list(pos)
    speech_parts_idx_list = ea.get_idx_list(pos_list, pos)
    consecutives_idx_list = ea.get_idx_list(consecutives_list, (pos, 1))
    for speech_part_idx, consecutives_idx in zip(speech_parts_idx_list, consecutives_idx_list):
        if consecutives_idx+1 < len(consecutives_list):
            o_count = consecutives_list[consecutives_idx+1][1]
            if pos == 'H-RQ':
                o_list = []
                for child_str in child_strs_list[speech_part_idx:speech_part_idx+o_count+1]:
                    if child_str not in ha.NAVIGABLE_PARENT_IS_QUAL_DICT:
                        o_list.append(child_str)
                    elif ha.NAVIGABLE_PARENT_IS_QUAL_DICT[child_str]:
                        o_list.append(child_str)
            else:
                 o_list = child_strs_list[speech_part_idx:speech_part_idx+o_count+1]
            section_list.extend(o_list)
print(consecutives_list)
#break

[('O', 5), ('H-LN', 1), ('O', 1), ('H-LN', 1), ('O', 7), ('H-RQ', 1), ('O', 11), ('H-LN', 1), ('O', 1), ('H-SP', 1), ('O', 11)]


In [11]:

for pos in list(set(pos_list) - set(['O'])):
    print(pos, hc.POS_EXPLANATION_DICT[pos], ea.get_idx_list(pos_list, pos))

H-LN Legal Notifications Header [5, 7, 27]
H-RQ Required Qualifications Header [15]
H-SP Supplemental Pay Header [29]


In [40]:

from random import shuffle

POPABLE_FILES_LIST = os.listdir(ha.SAVES_HTML_FOLDER)
shuffle(POPABLE_FILES_LIST)
s.store_objects(POPABLE_FILES_LIST=POPABLE_FILES_LIST)

In [90]:

ZMQInteractiveShell_obj = get_ipython()
NAVIGABLE_PARENT_IS_QUAL_DICT = s.load_object('NAVIGABLE_PARENT_IS_QUAL_DICT')
def get_pos_code():
    POPABLE_FILES_LIST = s.load_object('POPABLE_FILES_LIST')
    file_name = POPABLE_FILES_LIST.pop()
    s.store_objects(POPABLE_FILES_LIST=POPABLE_FILES_LIST)
    NAVIGABLE_PARENT_IS_QUAL_DICT = s.load_object('NAVIGABLE_PARENT_IS_QUAL_DICT')
    output_str = f'# {file_name}\n'
    EXTRACTED_EDUCATION_LIST = []
    EXTRACTED_PREF_QUALS_LIST = []
    EXTRACTED_TASK_SCOPE_LIST = []
    EXTRACTED_INTERVIEW_LIST = []
    EXTRACTED_LEGAL_LIST = []
    EXTRACTED_CORP_LIST = []
    EXTRACTED_OTHER_LIST = []
    (extracted_posting_location, EXTRACTED_LOCATION_LIST, EXTRACTED_REQ_QUALS_LIST, EXTRACTED_PAY_LIST, EXTRACTED_DURATION_LIST,
     extracted_job_title_list) = extract_job_title_info(file_name)
    pos_list, child_strs_list = get_pos_list(file_name)
    consecutives_list = []
    for key_pos, consecutive_value in groupby(pos_list):
        value_list = list(consecutive_value)
        consecutives_count = len(value_list)
        consecutives_tuple = (key_pos, consecutives_count)
        consecutives_list.append(consecutives_tuple)
    for pos in list(set(pos_list) - set(['O'])):
        section_list = get_section_list(pos)
        speech_parts_idx_list = ea.get_idx_list(pos_list, pos)
        consecutives_idx_list = ea.get_idx_list(consecutives_list, (pos, 1))
        for speech_part_idx, consecutives_idx in zip(speech_parts_idx_list, consecutives_idx_list):
            if consecutives_idx+1 < len(consecutives_list):
                o_count = consecutives_list[consecutives_idx+1][1]
                if pos == 'H-RQ':
                    o_list = []
                    for child_str in child_strs_list[speech_part_idx:speech_part_idx+o_count+1]:
                        if child_str not in ha.NAVIGABLE_PARENT_IS_QUAL_DICT:
                            o_list.append(child_str)
                        elif ha.NAVIGABLE_PARENT_IS_QUAL_DICT[child_str]:
                            o_list.append(child_str)
                else:
                     o_list = child_strs_list[speech_part_idx:speech_part_idx+o_count+1]
                section_list.extend(o_list)
        if len(section_list):
            output_str += '\n'
            output_str += '# '
            output_str += pos
            output_str += ' '
            output_str += hc.POS_EXPLANATION_DICT[pos]
            output_str += '\n'
            output_str += 'for tag_str in [\n'
            for child_str in section_list:
                if child_str not in NAVIGABLE_PARENT_IS_QUAL_DICT:
                    if "'" in child_str:
                        output_str += f'        "{child_str}",\n'
                    else:
                        output_str += f"        '{child_str}',\n"
            output_str += '    ]:\n'
            if pos in ['H-RQ', 'H-PQ']:
                output_str += '    NAVIGABLE_PARENT_IS_QUAL_DICT[tag_str] = True\n'
            else:
                output_str += '    NAVIGABLE_PARENT_IS_QUAL_DICT[tag_str] = False\n'
            output_str += '''print(len(NAVIGABLE_PARENT_IS_QUAL_DICT.keys()))
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DICT=NAVIGABLE_PARENT_IS_QUAL_DICT)\n'''
    
    return output_str


---
# Training

In [ ]:

ZMQInteractiveShell_obj.set_next_input(text=get_pos_code(), replace=True)


---
# Inference


----
# Clean up some missing data

In [ ]:

child_strs_list

In [91]:

child_str = '<b>About the Team</b>'
child_str in ha.NAVIGABLE_PARENT_IS_HEADER_DICT

In [92]:

ha.NAVIGABLE_PARENT_IS_HEADER_DICT[child_str] = True
s.store_objects(NAVIGABLE_PARENT_IS_HEADER_DICT=ha.NAVIGABLE_PARENT_IS_HEADER_DICT)

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\NAVIGABLE_PARENT_IS_HEADER_DICT.pickle


In [93]:

child_str in hc.CORPORATE_SCOPE_HEADERS_LIST

In [94]:

hc.CORPORATE_SCOPE_HEADERS_LIST.append(child_str)
s.store_objects(CORPORATE_SCOPE_HEADERS_LIST=hc.CORPORATE_SCOPE_HEADERS_LIST)

Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\CORPORATE_SCOPE_HEADERS_LIST.pickle


In [89]:
# Software_Engineer_-_Machine_Learning_-_Remote_-_Indeed.com_f6b4585215cfe949.html

# H-TS Task Scope Header
for tag_str in [
        '<h2 class="jobsearch-JobDescriptionSection-jobDescriptionTitle icl-u-xs-my--md" id="jobDescriptionTitle">Full Job Description</h2>',
        "<p>O'Reilly Media is looking for a machine learning engineer to inform the direction and execution of our internal personalization and user tracking system built to support our learning platform.</p>",
    ]:
    NAVIGABLE_PARENT_IS_QUAL_DICT[tag_str] = False
print(len(NAVIGABLE_PARENT_IS_QUAL_DICT.keys()))
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DICT=NAVIGABLE_PARENT_IS_QUAL_DICT)

# H-RQ Required Qualifications Header
for tag_str in [
        '<b>Requirements</b>',
        "<li>A bachelor's degree in a math focused discipline, or equivalent real world experience</li>",
        '<li>Experience working with a modern data science / statistics platform such as scikit, R, SAS, matlab or similar.</li>',
        '<li>Experience with how to design features for use in a linear model and/or a neural network to achieve specific predictive outcomes.</li>',
        '<li>Previous experience working with an object oriented programming language such as Java or Scala.</li>',
        '<li>The extreme desire to learn and solve problems you have never encountered before.</li>',
        '<b>Preferred skills</b>',
        '<li>Experience with big data systems like Hadoop, Spark, Beam, HBase, etc.</li>',
        '<li>Strong data structure and algorithm knowledge</li>',
        '<li>TensorFlow / DNNs and Language Models</li>',
    ]:
    NAVIGABLE_PARENT_IS_QUAL_DICT[tag_str] = True
print(len(NAVIGABLE_PARENT_IS_QUAL_DICT.keys()))
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DICT=NAVIGABLE_PARENT_IS_QUAL_DICT)

# H-LN Legal Notifications Header
for tag_str in [
        '<b>About the Team</b>',
        "<p>O’Reilly Media’s Search and Personalization Platform Engineering team supports O’Reilly’s premier learning service by building out personalization systems built on data and machine learning to provide a unique and dynamic experience for all of our users. The O'Reilly Learning Platform is used by technologists, managers, and designers around the world to hone their skills and improve their craft. O'Reilly's personalization team is one of many small teams that are broadly distributed across the US, featuring diverse, tightly collaborative groups of developers, designers, and product managers constantly encouraging each other to deliver work that instills pride and fulfillment. We encourage learning, knowledge sharing, growth and collaboration in all aspects.</p>",
        '<b>About the Job</b>',
        '<p>At O’Reilly our engineers focus on building a diverse set of features designed in collaboration with product managers, UX and other teammates. Engineers work with product managers to refine direction and solve user problems, exchange code reviews with other team members, provide mentorship to junior engineers, and assist QA and Ops in troubleshooting product issues.</p>',
        '<p>In this role you will be primarily building data processing systems and developing machine learning models to provide intelligent predictions that are used within the search and personalization team or other internal teams to drive improved learning outcomes across the platform. In addition to building models, you will also be responsible for building and maintaining web services that provide search and personalization services to our users. This can include building apis, managing search engines, as well as building any other systems required to meet our goals.</p>',
        '<b>About You</b>',
        '<p>We are interested in people who have experience building and supporting web applications with a diverse and engaged user base. We desire candidates who work comfortably in an agile environment and with collaborators who are distributed across multiple time zones. We value colleagues who are helpful, respectful, humble, and always willing to do what’s best for our users. We desire developers who treat automated tests as essential, and believe that code reviews are a crucial path of learning and of sharing knowledge. The women and men of our platform team have taken many traditional and nontraditional paths to the developer profession, and we welcome diverse teams that are bound together by a mutual love of learning.</p>',
        '<b>Bonus skills</b>',
        '<li>Finagle, GCP, Apache Beam</li>',
        '<li>Docker + Kubernetes</li>',
        '<b>About the Company</b>',
        '<p>O’Reilly Media has been inspiring the future for more than 40 years!</p>',
        '<p>We share the knowledge and teach the skills people need to change their world. For more than 40 years, O’Reilly has imparted the world-shaping ideas of innovators through books, articles, conferences, and our online learning platform.</p>',
        "<p>When individuals, teams, and entire enterprises connect with the world's leading experts and content providers, anything is possible. Whether you're working to advance your career, be a better manager, or achieve the next breakthrough in technology or business, learning new skills is at the heart of it all.</p>",
        '<p>With a range of formats including live online training courses, interactive tutorials, books, videos, and case studies, we equip all members of the workforce with the insight they need to stay ahead in an ever-changing economy.</p>',
        '<p>Learning should not be measured in terms of usage, course completion, or certifications. Learning should be measured in terms of outcomes. We want to prepare people to solve challenging problems and inspire them with what’s possible for the future.</p>',
        '<p>Something we’ve been doing for over 40 years.</p>',
    ]:
    NAVIGABLE_PARENT_IS_QUAL_DICT[tag_str] = False
print(len(NAVIGABLE_PARENT_IS_QUAL_DICT.keys()))
s.store_objects(NAVIGABLE_PARENT_IS_QUAL_DICT=NAVIGABLE_PARENT_IS_QUAL_DICT)


Pickling to C:\Users\dev\Documents\Repositories\job-hunting\saves\pickle\POPABLE_FILES_LIST.pickle
